## How to manually apply windowing to an x-ray image.

- In this notebook, we'll grab an image from the SIIM Covid19 dataset and manually apply a Window Width and Window Level to it.

Normally, the pixel ranges of x-rays are much larger than the pixel display range of consumer grade monitors. 10-16 bit images on 8 bit displays.

Since we can only display 256 shades of gray (8 bit) on a most monitors, we must map *many-to-one* pixels here using a Look Up Table (LUT).

Automatically calculating the width/center does not always produce the best diagnostic quality image for the pathology in question.

DICOM image viewers typically have a tool for windowing that allows radiologists to adjust the levels from the full image, rather than the 8 bit representation of it. This allows them to see subtle differences that they would not otherwise be able to see.

Setting the Window Width and Window Level creates a VOI (Values of Interest) LUT and applies it to the image. This is a lossy operation.

Some DICOM images contain VOI LUT arrays, reference external VOI LUTs, or use specified Width/Level tags to calculate the LUT.

** VOI LUT should not be confused with Modality LUT, which maps modality unit specific values to usable pixel values. If a modality LUT is present in the DICOM image, it should always be applied first in the pipeline.*

- Here's a couple DICOM pre-processing notebooks I made:
- Rib supression on Chest X-Rays -> https://www.kaggle.com/davidbroberts/rib-suppression-poc
- Apply Unsharp Mask to Chest X-Rays -> https://www.kaggle.com/davidbroberts/unsharp-masking-chest-x-rays
- Cropping Chest X-Rays -> https://www.kaggle.com/davidbroberts/cropping-chest-x-rays
- Visualizing Chest X-Ray bit planes -> https://www.kaggle.com/davidbroberts/visualizing-chest-x-ray-bitplanes
- DICOM full range pixels as CNN input -> https://www.kaggle.com/davidbroberts/dicom-full-range-pixels-as-cnn-input

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import pydicom
import matplotlib.pyplot as plt
%matplotlib inline

### Create a couple helper functions

In [ ]:
# This function gets the first image path in a StudyInstanceUID directory
def get_image_by_study_id(study_id):
    base_path = "/kaggle/input/siim-covid19-detection/"
    study_path = base_path + "train/" + study_id + "/"
    images = []
    for subdir, dirs, files in os.walk(study_path):
        for file in files:     
            image = os.path.join(subdir, file)
            if os.path.isfile(image):
                return image
    return "none"

In [ ]:
# Make a simple linear VOI LUT from the raw (stored) pixel data
def make_lut(storedPixels, windowWidth, windowLevel, p_i):
    
    # Slope and Intercept set to 1 and 0 for X-ray. Get these from DICOM tags instead if using 
    # on a modality that requires them (CT, PT etc)
    slope = 1.0
    intercept = 0.0
    minPixel = int(np.amin(storedPixels))
    maxPixel = int(np.amax(storedPixels))

    # Make an empty array for the LUT the size of the pixel 'width' in the raw pixel data
    lut = [0] * (maxPixel + 1)
    
    # Invert pixels and windowLevel for MONOCHROME1. We invert the specified windowLevel so that 
    # increasing the level value makes the images brighter regardless of photometric intrepretation
    invert = False
    if p_i == "MONOCHROME1":
        invert = True
    else:
        windowLevel = (maxPixel - minPixel) - windowLevel
        
    # Loop through the pixels and calculate each LUT value
    for storedValue in range(minPixel, maxPixel):
        modalityLutValue = storedValue * slope + intercept
        voiLutValue = (((modalityLutValue - windowLevel) / windowWidth + 0.5) * 255.0)
        clampedValue = min(max(voiLutValue, 0), 255)
        if invert:
            lut[storedValue] = round(255-clampedValue)
        else:
            lut[storedValue] = round(clampedValue)
        
    return lut

In [ ]:
# Apply the LUT to a pixel array
def apply_lut(pixels_in, lut):
    pixels_in = pixels_in.flatten()
    pixels_out = [0] * len(pixels_in)
    for i in range(0, len(pixels_in)):
        pixel = pixels_in[i]
        pixels_out[i] = int(lut[pixel])
    return pixels_out

### Display an image with default windowing.

In [ ]:
# Take a look at a random image and display it with imshow. The width and level are automagically calculated from the pixel data.
img_file = get_image_by_study_id("00c241c3fc0d")
print("Loading image: " + img_file)

img = pydicom.dcmread(img_file)
plt.imshow(img.pixel_array,cmap="gray");

### Calculate the width and level from the pixel ranges manually.

In [ ]:
# Set the width to be the distance between the pixels, and the level to be the middle of the pixel range.
# The resulting image should look exactly like the one above.
pixels = img.pixel_array
minPixel = np.min(pixels)
maxPixel = np.max(pixels)
windowWidth = maxPixel - minPixel
windowLevel = (minPixel + maxPixel) / 2

lut = make_lut(pixels, windowWidth, windowLevel, img.PhotometricInterpretation)
pixels = apply_lut(pixels, lut)

# Reshape the pixel array back into the image shape
img_out = np.reshape(pixels, (img.pixel_array.shape[0],img.pixel_array.shape[1]))

print("Calculated - Width: " + str(windowWidth) + " / Level  " +  str(windowLevel))
print("Pixel range: " + str(minPixel) + " - " + str(maxPixel))
plt.imshow(img_out,cmap="gray");

- In this case, the image pixel range is 0-4095, which indicates this is probably a 12 bit image.
- Since there isn't a 12 bit data type, we have to store these in 16 bit arrays.
- The Photometric Interpretation is MONOCHROME2, meaning the pixel intensities get brighter as the pixel value increases.
- MNONOCHROME1 images, the intensities get brighter as the pixel value decreases.
- We can verify by looking at a couple DICOM tags.

In [ ]:
print("PhotometricInterpretation: " + img.PhotometricInterpretation)
print("Bit stored: " + str(img.BitsStored))

### Manually specify window width and level.

In [ ]:
# If we use windowWidth = 1000, and keep the Level at center .. the image will no longer be 'full width' 
# Setting the width to less than the distance between the output pixels means the full range of pixels 
# is not used. The resulting image displays less than 255 shades of grey. If the width was set to '2',
# the image would only display black or white pixels.

pixels = img.pixel_array
windowWidth = 1000
windowLevel = 2047

lut = make_lut(pixels, windowWidth, windowLevel, img.PhotometricInterpretation)
pixels = apply_lut(pixels, lut)

print("Width: " + str(windowWidth) + " / Level  " +  str(windowLevel))
img_out = np.reshape(pixels, (img.pixel_array.shape[0],img.pixel_array.shape[1]))
plt.imshow(img_out,cmap="gray");

In [ ]:
# Let's make this image full width, but a little brighter

pixels = img.pixel_array
windowWidth = 4095
windowLevel = 3500

lut = make_lut(pixels, windowWidth, windowLevel, img.PhotometricInterpretation)
pixels = apply_lut(pixels, lut)

print("Manual Width: " + str(windowWidth) + " / Level  " +  str(windowLevel))
img_out = np.reshape(pixels, (img.pixel_array.shape[0],img.pixel_array.shape[1]))
plt.imshow(img_out,cmap="gray");

### Conclusion
- Creating image sets with various Widths and Levels might offer more 'visible' information when exporting to 8 bit for model import.
- You can try various width/level settings to best demonstrate the pathology in question.